# Apriori Algorithm for Associative Rule Mining

In [ ]:
#The objective is to develop rules involving regions, castes and coverage with adequate water supply
#The data points are initially converted into transactions using discretization for the continuous variables
#The Apriori Algorithm is used to generate frequent itemsets along with their support count
#Rules are mined by evalutaing confidence of binary partitions of the subsets 
#Controllable parameters: 
#     min_pop : Minimum population of caste for datapoint
#     min_pop_percentage : Minimum population percentage of caste for datapoint
#     Discretization limits : Necessary to categorize values for discretization
#     min_sup : Minimum Support Value
#     min_conf : Minimum Confidence Value
    
    

In [456]:
import pandas as pd
import numpy as np
from itertools import combinations, chain

data = pd.read_csv('/Users/GJ/Desktop/DM_EndSem/final_dataset_for_2009.csv')#This dataset is the file produced after preprocessing the original data


## Preparing Dataset for Conversion 

In [457]:
df = pd.DataFrame(data)
#Allocating region to each state in order to get better readability while analysing the obtained rules
dictionary = {'andhra pradesh': 'S','arunachal pradesh': 'E','assam': 'E','bihar': 'C','chandigarh': 'N',
  'chhattisgarh': 'C','dadra & nagar haveli': 'W','daman & diu':'W','goa': 'W','national capital territory of delhi': 'N',
  'gujarat': 'W','haryana': 'N','himachal pradesh': 'N','jammu and kashmir': 'N','jharkhand': 'E','karnataka': 'S',
  'kerala': 'S','ladakh': 'N','lakshadweep': 'S','madhya pradesh': 'C','maharashtra': 'S','manipur': 'E','meghalaya': 'E',
  'mizoram': 'E','nagaland': 'E','orissa': 'E','punjab': 'N','puducherry':'S','rajasthan': 'W','sikkim': 'E',
  'tamil nadu': 'S','telangana': 'S','tripura': 'E','uttar pradesh': 'N','uttarakhand': 'N','west bengal': 'W'}

df['Region_New'] = df['State Name'].map(dictionary) #Mapping to region name based on state
df = df.drop(columns=['District Name','Block Name','Region','State Name']) #Dropping unnecessary attributes
df.head()
df.tail()

4907


,SC Current Population,ST Current Population,GENERAL Current Population,State Name,SC percent covered,ST percent covered,GENERAL percent covered,SC Population Percentage,ST Population Percentage,GENERAL Population Percentage,Region_New
4867,19633,526,158351,bihar,91.570315,73.954373,72.429603,10.998263,0.294661,88.707075,C
4868,8394,165918,34288,madhya pradesh,68.763402,56.876891,73.792580,4.023969,79.538830,16.437200,C
4869,0,12776,329,arunachal pradesh,84.348722,37.343456,0.000000,0.000000,97.489508,2.510492,E
4870,24,16900,536,arunachal pradesh,37.500000,32.881657,27.611940,0.137457,96.792669,3.069874,E
4871,0,12256,374,nagaland,84.348722,66.465405,99.465241,0.000000,97.038797,2.961203,E


In [ ]:
#df.describe() #To estimate minimum population and population percentage

## Conversion of Dataset into Transaction Dataset 

In [458]:
#The dataset set is converted to a transaction dataset in order to procede with the Apriori Algorithm
Transactions = []#This list will contain the set of all transactions
i = 0 
min_pop = 100 #The minimum population required for caste to be considered in order to ignore small sample sizes
min_pop_percetage = 0 #The minimum population percentage required for caste to be considered in order to ignore 
                       #highly disproportionate cases.
df_size = len(df.index)# Size of Dataset
while i<df_size:
    T = []
    T.append(df.loc[i,'Region_New']) # The first element of each transaction is the region name
    c11 = df.loc[i,'SC Current Population']#Values of population, population percentage, covered population percentage
    c21 = df.loc[i,'ST Current Population']#are extracted for the current element for each caste
    c31 = df.loc[i,'GENERAL Current Population']
    c12 = df.loc[i,'SC Population Percentage']
    c22 = df.loc[i,'ST Population Percentage']
    c32 = df.loc[i,'GENERAL Population Percentage']
    c13 = df.loc[i,'SC percent covered'] 
    c23 = df.loc[i,'ST percent covered'] 
    c33 = df.loc[i,'GENERAL percent covered']

    D1 = 80 #These are the boundaries for classifying a point into high,medium or low water coverage
    D2 = 40
    if c11>min_pop and c12>min_pop_percetage:
        if c13>D1:
            T.append('SC_CH')#This implies the datapoint has high population of SC covered with adequate water
        else: 
            if c13>D2:
                T.append('SC_CM')#This implies the datapoint has medium population of SC covered with adequate water
            else:
                if c13!=0:
                    T.append('SC_CL')#This implies the datapoint has low population of SC covered with adequate water  
    if c21>min_pop and c22>min_pop_percetage:
        if c23>D1:
            T.append('ST_CH')#This implies the datapoint has high population of ST covered with adequate water
        else: 
            if c23>D2:
                T.append('ST_CM')#This implies the datapoint has medium population of ST covered with adequate water
            else:
                if c23!=0:
                    T.append('ST_CL')#This implies the datapoint has low population of ST covered with adequate water
    if c31>min_pop and c32>min_pop_percetage:
        if c33>D1:
            T.append('GN_CH')#This implies the datapoint has high population of GENERAL covered with adequate water
        else: 
            if c33>D2:
                T.append('GN_CM')#This implies the datapoint has medium population of GENERAL covered with adequate water
            else:
                if c33!=0:
                    T.append('GN_CL')#This implies the datapoint has low population of GENERAL covered with adequate water 
    i = i+1            
    Transactions.append(T)
#Printing the first and last five elements of transaction dataset
i = 0
while i<5:
    print(Transactions[i])
    i = i+1
i = 1
print(" ")
while i<=5:
    print(Transactions[df_size-i])
    i = i+1


['E', 'ST_CH', 'GN_CL']
['S', 'SC_CH', 'ST_CH', 'GN_CH']
['W', 'SC_CH', 'ST_CH', 'GN_CH']
['N', 'SC_CH', 'GN_CH']
['W', 'SC_CH', 'ST_CH', 'GN_CH']
 
['E', 'ST_CM', 'GN_CH']
['E', 'ST_CL', 'GN_CL']
['E', 'ST_CL']
['C', 'SC_CM', 'ST_CM', 'GN_CM']
['C', 'SC_CH', 'ST_CM', 'GN_CM']


## Frequent Itemset Generation from Transaction Data 

In [459]:
# # Frequent Itemset Generation

def Support_Counting(T,C):
    Supp = []
    for x in C:
        i = 0
        s = 0
        while i<df_size:
            if (set(x).issubset(set(T[i])))==True:
                s = s+1
            i = i+1
        Supp.append(s)  
    return Supp
    
def Candidate_Generation(List,k):
    C = ['N','S','E','W','C','SC_CH','SC_CM','SC_CL','ST_CH','ST_CM','ST_CL','GN_CH','GN_CM','GN_CL']
    if k == 1:
        C = [['N'],['S'],['E'],['W'],['C'],['SC_CH'],['SC_CM'],['SC_CL'],['ST_CH'],['ST_CM'],['ST_CL'],['GN_CH'],
             ['GN_CM'],['GN_CL']]
        return C
    else:
        L = list(map(set, itertools.combinations(C,k)))
        return L

def Support_Pruning(C,S):
    min_sup = 100
    I = []
    F = []
    FS = []
    for x in C:
        i = C.index(x)
        if S[i]<min_sup:
            I.append(list(x))
        else:
            F.append(list(x))
            FS.append(S[i])
    return F,FS,I

def Subset_Pruning(C,I):
    if I!=[]:
        Ind = I
        for x in I:
            for y in C:
                if (set(x).issubset(y))==True:
                    Ind.pop(Ind.index(x))
                    break
    return C

k = 1
Cand = [] #Corresponds to all candidates generated i.e. Ci
Infreq_Cand = [] #Correspods to candidates which were pruned 
List = [] #Corresponds to set of frequent candidates i.e Li
Support = [] #Corresponds to a list containing support values reflecting all elemnts in List

while k<=4: #Candidate generation is required only till size 4 because the maximum size of any transaction is 4
    C = Candidate_Generation(List,k) # This function generates all possible candidates of size k 
    if k!=1:
        C = Subset_Pruning(C,Infreq_Cand) #This function prunes the generated candidates if any subset is infrequent
    Cand.append(C)
    S = Support_Counting(Transactions,C) #This function calculates the support for the remaining candidates
    L,S,I = Support_Pruning(C,S) # This function prunes the canidadtes based on support count
    List.append(L) #The remaining candidates are added to the frequent itemset list
    Support.append(S) #The support corresponding to each candidate is added to this list in the same position as the candidate
    if I!=[]:
        for x in I:
            Infreq_Cand.append(x) # Infrequent itemsets are collected in this list which are used in "subset_pruning" function
    k = k+1
i = 0
while i<4:
    print("Frequent Itemsets of size - ",i+1,", number of elements - ",len(List[i]),"\n",List[i],"\n")
    print("Corredsponding Support: \n",Support[i])
    print(" ")
    i=i+1

Frequent Itemsets of size -  1 , number of elements -  14 
 [['N'], ['S'], ['E'], ['W'], ['C'], ['SC_CH'], ['SC_CM'], ['SC_CL'], ['ST_CH'], ['ST_CM'], ['ST_CL'], ['GN_CH'], ['GN_CM'], ['GN_CL']] 

Corredsponding Support: 
 [731, 2093, 779, 455, 814, 3161, 1229, 187, 2202, 1224, 212, 2999, 1524, 232]
 
Frequent Itemsets of size -  2 , number of elements -  38 
 [['N', 'SC_CH'], ['SC_CM', 'N'], ['GN_CH', 'N'], ['GN_CM', 'N'], ['SC_CH', 'S'], ['SC_CM', 'S'], ['ST_CH', 'S'], ['ST_CM', 'S'], ['GN_CH', 'S'], ['GN_CM', 'S'], ['SC_CH', 'E'], ['SC_CM', 'E'], ['ST_CH', 'E'], ['ST_CM', 'E'], ['GN_CH', 'E'], ['GN_CM', 'E'], ['W', 'SC_CH'], ['ST_CH', 'W'], ['GN_CH', 'W'], ['C', 'SC_CH'], ['C', 'SC_CM'], ['ST_CH', 'C'], ['ST_CM', 'C'], ['GN_CH', 'C'], ['GN_CM', 'C'], ['ST_CH', 'SC_CH'], ['ST_CM', 'SC_CH'], ['GN_CH', 'SC_CH'], ['GN_CM', 'SC_CH'], ['ST_CH', 'SC_CM'], ['ST_CM', 'SC_CM'], ['GN_CH', 'SC_CM'], ['GN_CM', 'SC_CM'], ['SC_CL', 'GN_CL'], ['GN_CH', 'ST_CH'], ['ST_CH', 'GN_CM'], ['ST_CM', 'GN_CH

## Mining Association Rules

In [462]:
from itertools import chain, combinations
import copy

def Generate_Subsets(S):
    s = list(S)
    A = chain.from_iterable(combinations(s, r) for r in range(1, len(s)))  
    A = [list(i) for i in A]
    return A

i = 2
CR1 = [] #list which will contain LHS of mined rules
CR2 = [] #list which will contain RHS of mined rules
CVal = [] #list which will contain confidence value of mined rules
min_conf = 0.95

while i<=4: #Rule generation is done iteratively for itemsets of different sizes
    L = List[i-1] #List of frequent itemsets of size (i-1)
    L1 = copy.deepcopy(L) #Used in order to prevent the altering of original list
    S = Support[i-1] #Support of frequent itemsets of size(i-1)
    while L1!=[]:
        L2 = L1[0] 
        L3 = Generate_Subsets(L2) #Function generates all possible subsets of L2 of size 1-(n-1) where n = len(L2)
        S1 = S[L.index(L2)] #Support value corresponding to L2
        for A in L3: #for ever subset
            G = List[len(A)-1]
            if G.count(A)==1:
                n = len(A)
                Stemp = Support[n-1]
                pos = List[n-1].index(A)
                S2 = Stemp[pos] #calculates support value of subset
                if (S1/S2)>min_conf: #calculates confidence by S1/S2
                    B = list(set(L2)-set(A)) #the complementary subset is appended to CR2
                    CR1.append(A)
                    CR2.append(B)
                    CVal.append(S1/S2)
        L1.pop(0)
    i=i+1
i = 0
while i <len(CR1):
    print(CR1[i],"->",CR2[i],";Confidence=",CVal[i])
    i = i+1


  

['GN_CH', 'N'] -> ['SC_CH'] ;Confidence= 0.9609665427509294
['N', 'SC_CH'] -> ['GN_CH'] ;Confidence= 0.9904214559386973
['ST_CH', 'S'] -> ['SC_CH'] ;Confidence= 0.9511844938980617
['GN_CH', 'S'] -> ['SC_CH'] ;Confidence= 0.9738717339667459
['SC_CH', 'S'] -> ['GN_CH'] ;Confidence= 0.9686946249261665
['ST_CH', 'W'] -> ['SC_CH'] ;Confidence= 0.9642857142857143
['GN_CH', 'W'] -> ['SC_CH'] ;Confidence= 0.9794721407624634
['W', 'SC_CH'] -> ['GN_CH'] ;Confidence= 0.9681159420289855
['GN_CH', 'ST_CH'] -> ['SC_CH'] ;Confidence= 0.960313315926893
['GN_CH', 'ST_CH', 'S'] -> ['SC_CH'] ;Confidence= 0.9825095057034221
['SC_CH', 'ST_CH', 'S'] -> ['GN_CH'] ;Confidence= 0.9750943396226415
['GN_CH', 'ST_CH', 'W'] -> ['SC_CH'] ;Confidence= 0.9863013698630136
['ST_CH', 'W', 'SC_CH'] -> ['GN_CH'] ;Confidence= 0.9696969696969697
